In [12]:
import os
os.chdir("C:/Users/mcnal/OneDrive/Documents/Personal Projects/housing_dashboard")

In [49]:
import json
import requests
import pandas as pd
import os
import csv

In [43]:
acs_year = "2018"
acs_type = "acs5"
acs_var = "B25002_001E,B25002_002E,B25002_003E"
acs_geolevel = "county"
acs_geo = "*"
good_url = "https://api.census.gov/data/2018/acs/acs5?get=NAME,group(B25024)&for=county"

In [44]:
url = "https://api.census.gov/data/{}/acs/{}?get=NAME,{}&for={}:{}".format(acs_year, acs_type, acs_var, acs_geolevel, acs_geo)

print(url)

https://api.census.gov/data/2018/acs/acs5?get=NAME,B25002_001E,B25002_002E,B25002_003E&for=county:*


In [45]:
acs_five = requests.get(url)

In [46]:
acs_five.status_code

200

In [47]:
acs5_county = json.loads(acs_five.text)

In [48]:
print(acs5_county)

[['NAME', 'B25002_001E', 'B25002_002E', 'B25002_003E', 'state', 'county'], ['Washington County, Mississippi', '21591', '18299', '3292', '28', '151'], ['Perry County, Mississippi', '5620', '4563', '1057', '28', '111'], ['Choctaw County, Mississippi', '4200', '3164', '1036', '28', '019'], ['Itawamba County, Mississippi', '10289', '8706', '1583', '28', '057'], ['Carroll County, Mississippi', '5154', '3658', '1496', '28', '015'], ['Grenada County, Mississippi', '10240', '8224', '2016', '28', '043'], ['Jefferson County, Mississippi', '3744', '2530', '1214', '28', '063'], ['Greene County, Mississippi', '5216', '4172', '1044', '28', '041'], ['Marshall County, Mississippi', '15474', '13074', '2400', '28', '093'], ['Quitman County, Mississippi', '3595', '3055', '540', '28', '119'], ['Bolivar County, Mississippi', '14362', '12159', '2203', '28', '011'], ['Lamar County, Mississippi', '24580', '22213', '2367', '28', '073'], ['Lee County, Mississippi', '36678', '32267', '4411', '28', '081'], ['Pike

In [61]:
df1 = pd.DataFrame(acs5_county[1:],columns=acs5_county[0])

In [65]:
print(df1)

                                NAME B25002_001E B25002_002E B25002_003E  \
0     Washington County, Mississippi       21591       18299        3292   
1          Perry County, Mississippi        5620        4563        1057   
2        Choctaw County, Mississippi        4200        3164        1036   
3       Itawamba County, Mississippi       10289        8706        1583   
4        Carroll County, Mississippi        5154        3658        1496   
...                              ...         ...         ...         ...   
3215            Clayton County, Iowa        9099        7652        1447   
3216        Buena Vista County, Iowa        8296        7550         746   
3217            Guthrie County, Iowa        5786        4397        1389   
3218           Humboldt County, Iowa        4729        4221         508   
3219         Washington County, Iowa        9686        8720         966   

     state county  
0       28    151  
1       28    111  
2       28    019  
3      

In [66]:
col_head = list(df1)

In [67]:
with open('variable_lookup.csv', mode='r') as infile:
    reader = csv.reader(infile)
    with open('variable_lookup_new.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        mydict = {rows[0]:rows[1] for rows in reader}

In [68]:
print(mydict)

{'ï»¿Variable ID': 'Variable Name', 'NAME': 'County', 'B25002_001E': 'Total Housing Units', 'B25002_002E': 'Occupied Units', 'B25002_003E': 'Vacant Units', 'state': 'State ID', 'county': 'County ID'}


In [69]:
new_col = [mydict.get(item,item) for item in col_head]

In [70]:
print(new_col)

['County', 'Total Housing Units', 'Occupied Units', 'Vacant Units', 'State ID', 'County ID']


In [75]:
df1.columns = new_col

In [80]:
df1['Vacant Units'] = pd.to_numeric(df1['Vacant Units'], errors='coerce')

In [81]:
df1['Total Housing Units'] = pd.to_numeric(df1['Total Housing Units'], errors='coerce')

In [82]:
df1['Occupied Units'] = pd.to_numeric(df1['Occupied Units'], errors='coerce')

In [83]:
df1['Vacancy Rate'] =  (df1['Vacant Units'] / df1['Total Housing Units'] ) * 100

In [84]:
print(df1)

                              County  Total Housing Units  Occupied Units  \
0     Washington County, Mississippi                21591           18299   
1          Perry County, Mississippi                 5620            4563   
2        Choctaw County, Mississippi                 4200            3164   
3       Itawamba County, Mississippi                10289            8706   
4        Carroll County, Mississippi                 5154            3658   
...                              ...                  ...             ...   
3215            Clayton County, Iowa                 9099            7652   
3216        Buena Vista County, Iowa                 8296            7550   
3217            Guthrie County, Iowa                 5786            4397   
3218           Humboldt County, Iowa                 4729            4221   
3219         Washington County, Iowa                 9686            8720   

      Vacant Units State ID County ID  Vacancy Rate  
0             3292   

In [85]:
df1.to_csv('acs5_county.csv', encoding='utf-8', index=False)